In [1]:
import cv2
import os
import glob
import numpy as np

In [2]:
def letterbox(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True, stride=32):
    # Resize and pad image while meeting stride-multiple constraints
    shape = img.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:  # only scale down, do not scale up (for better test mAP)
        r = min(r, 1.0)

    # Compute padding
    ratio = r, r  # width, height ratios
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding
    if auto:  # minimum rectangle
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)  # wh padding
    elif scaleFill:  # stretch
        dw, dh = 0.0, 0.0
        new_unpad = (new_shape[1], new_shape[0])
        ratio = new_shape[1] / shape[1], new_shape[0] / shape[0]  # width, height ratios

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return img, ratio, (dw, dh)

In [3]:
def load_image(path, augment = False, img_size = 1024):

    img = cv2.imread(path)  # BGR
    assert img is not None, 'Image Not Found ' + path
    h0, w0 = img.shape[:2]  # orig hw
    r = img_size / max(h0, w0)  # resize image to img_size
    if r != 1:  # always resize down, only resize up if training with augmentation
        interp = cv2.INTER_AREA if r < 1 and not augment else cv2.INTER_LINEAR
        img = cv2.resize(img, (int(w0 * r), int(h0 * r)), interpolation=interp)
    return img, (h0, w0), img.shape[:2]  # img, hw_original, hw_resized
   

In [4]:
def resize_image(image_path):
    image_dir = '/home/hana/sonnh/kaggle-vin/kaggle-VinBigData-Chest-X-ray-Abnormalities-Detection/dataset/fold4/images/train_full_size'
    image_small_dir = '/home/hana/sonnh/kaggle-vin/kaggle-VinBigData-Chest-X-ray-Abnormalities-Detection/dataset/fold4/images/train'
    img,_, _ = load_image(os.path.join(image_dir, image_path), img_size = 960)
    cv2.imwrite(os.path.join(image_small_dir, image_path), img)

In [5]:
import os
from multiprocessing import Pool

pool = Pool()
image_dir = '/home/hana/sonnh/kaggle-vin/kaggle-VinBigData-Chest-X-ray-Abnormalities-Detection/dataset/fold4/images/train_full_size'
data = pool.map(resize_image, os.listdir(image_dir))